### Import data

In [1]:
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
def read_data(file_path):
    return pd.read_csv(file_path)

def get_isins(data):
    return data['Isin'].tolist()


def concatenate_data(prefixes, isindata, start_date, end_date):
    yf.pdr_override()
    data_list = []
    for prefix in prefixes:
        filtered_isin = []
        for i in range(0, len(isindata)):
            if isindata[i][:len(prefix)] == prefix:
                filtered_isin.append(isindata[i])
        data = pdr.get_data_yahoo(filtered_isin, start=start_date, end=end_date)
        data_list.append(data)
    combined_data = pd.concat(data_list, sort=False, axis=1, join='inner')
    return combined_data

def remove_null_columns(data):
    null_columns = []
    for col in data.columns:
        if data[col].isnull().all():
            null_columns.append(col)
    return data.drop(columns=null_columns)



def filter_by_column(data,stock_data, column_name, column_value=None):
    if column_value is not None:
        filtered = data[data[column_name] == column_value]
        data_isin = get_isins(filtered)
        filtered_data = [c for c in stock_data.columns if c in data_isin]
        subset = stock_data[filtered_data]
        return subset
    else:
        return stock_data
    

    
def filter_by_value(stock_data, data, column_name, threshold=None, operator=None):
    if threshold is not None:
        if operator == 'leq':
            filtered=data[data[column_name] <= threshold]
            data_isin = get_isins(filtered)
            filtered_data = [c for c in stock_data.columns if c in data_isin]
            subset = stock_data[filtered_data]
            return subset

        if operator == 'geq':
            filtered=data[data[column_name] >= threshold]
            data_isin = get_isins(filtered)
            filtered_data = [c for c in stock_data.columns if c in data_isin]
            subset = stock_data[filtered_data]
            return subset

        else:
            return stock_data
    else:
        return stock_data
    

def to_date(data,time='y'):
    adj_pct = data.ffill().pct_change()
    adj_pct.index = pd.to_datetime(adj_pct.index)
    result = adj_pct.resample(time).sum()
    return result


def get_stock_data(file_path, prefixes, start_date, end_date):
    if stock_data not in globals():
        data = read_data(file_path)
        isins = get_isins(data)
        st_data = concatenate_data(prefixes, isins, start_date, end_date)
        stock_data = remove_null_columns(st_data)
        return stock_data
    else:
        return stock_data


def get_filtered_stock_data(file_path, column_name, column_value, prefixes, start_date, end_date, time='y', threshold=None, operator=None):
    global stock_data
    
    if stock_data not in globals():
   
        data = read_data(file_path)
        
        isins = get_isins(data)
        
        stock_data = concatenate_data(prefixes, isins, start_date, end_date)
        
        stock_data = remove_null_columns(stock_data)
    
    stock_data = stock_data['Adj Close']
    
    if isinstance(threshold, int):
            
        filtered_data = filter_by_value(stock_data, data, column_name, threshold, operator)
            
    elif threshold is None:
            
        filtered_data = filter_by_column(data,stock_data, column_name, column_value)
            
    formatted_data = to_date(filtered_data, time)
    return formatted_data


In [3]:
%%time
stock_data = None
file_path = "ESG_DATA.csv"
column_name = 'environment_level'
column_value = 'Excellent'
threshold = None
operator = 'geq'
prefixes= ['SE', 'DK','NO']
start_date = '2010-01-01'
end_date = '2021-04-30'

CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
filtered_data = get_filtered_stock_data(file_path, column_name, column_value, prefixes, start_date, end_date, time='y', threshold=threshold, operator=operator)
filtered_data

[*********************100%***********************]  115 of 115 completed

10 Failed downloads:
- : No timezone found, symbol may be delisted
- COLL.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- ARPL.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- JOBS.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- BRILL.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- INT.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- VIVA.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- SFL.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- PURE.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- CTEK.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
[*********************100%***********************]  36 of 36 completed

1 Failed download:
- : No timezone found, symbol may be delisted
[***********

,SE0011337708,DK0010263722,DK0060094928
Date,,,
2010-12-31 00:00:00+01:00,0.233126,-0.301292,0.000000
2011-12-31 00:00:00+01:00,0.085138,-0.637522,0.000000
2012-12-31 00:00:00+01:00,0.402724,0.004191,0.000000
2013-12-31 00:00:00+01:00,0.441324,-0.208075,0.000000
2014-12-31 00:00:00+01:00,0.042047,0.036437,0.000000
2015-12-31 00:00:00+01:00,0.447685,-0.089911,0.000000
2016-12-31 00:00:00+01:00,-0.009339,0.083899,0.056770
2017-12-31 00:00:00+01:00,0.183381,-0.068273,0.276583
2018-12-31 00:00:00+01:00,0.077873,0.409519,0.299253


In [ ]:
data = read_data(file_path)
isins = get_isins(data)
stock_data = concatenate_data(prefixes, isins, start_date, end_date)
stock_data = remove_null_columns(stock_data)